In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from google.colab import drive
import os
import pickle
from scipy.signal import find_peaks, argrelextrema
from scipy.spatial.distance import cdist

In [ ]:
#Function that integrates for a single step by rk4
def RK4step(func, current_state, t, dt):
    k1 = func(t, current_state)
    k2 = func(t + 0.5 * dt, current_state + 0.5 * dt * k1)
    k3 = func(t + 0.5 * dt, current_state + 0.5 * dt * k2)
    k4 = func(t + dt, current_state + dt * k3)
    return current_state + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)

In [ ]:
# class that represents the lorenz system (drive system for the reservoir)
class LorenzSystem:

    def __init__(self):
        self.sigma = 10.0
        self.rho = 28.0
        self.beta = 8.0 / 3.0
        self.state = np.array([1, 1, 1])

    def __call__(self, t, u):
        return np.array([
            self.sigma * (u[1] - u[0]),
            u[0] * (self.rho - u[2]) - u[1],
            u[0] * u[1] - self.beta * u[2]
        ])

    def reinitialise_ic(self):
        self.state = np.array([1, 1, 1])

    def randomise_ic(self):
        self.state = 2 * (np.random.rand(self.state.size) - 0.5)

In [ ]:
# class that represents the RC object, this contains the response system inside of it, and it takes the drive system as an argument in it's initialisation.
# Note this uses np arrays for the matrices - it is to be used for smaller reservoir dimensional systems
class ReservoirComputer:
    def __init__(self, drive_system, reservoir_size, connectivity, spectral_radius, gamma, sigma):
        #initialise rng for each random element for repeatability
        self.rng_M = np.random.default_rng(1911)
        self.rng_Win = np.random.default_rng(1912)
        self.rng_state = np.random.default_rng(1913)

        #define all internal states, parameters and functions to be called on
        self.r = 2 * (self.rng_state.random(reservoir_size) - 0.5)
        self.u = drive_system
        self.spectral_radius = spectral_radius
        self.connectivity = connectivity
        self.reservoir_size = reservoir_size

        self.M, self.original_M, self.sparsification_mask = self.generate_M_and_mask(reservoir_size, connectivity, spectral_radius)
        self.Win = self.generate_Win(reservoir_size, self.u.state.size)

        self.sigma = sigma
        self.gamma = gamma
        self.res_update_func_listen = self.reservoir_update_equation
        self.res_update_func_pred = self.reservoir_update_equation_prediction

        self.dt = 0.01
        self.t = 0.0
        self.beta = 0.001
        self.q = lambda x: np.concatenate((np.array(x), np.array(x)**2))
        self.A = None
        self.b = None
        self.phi = None

    # Generate M by erdos-renyi topology and return mask so the topology can be saved
    def generate_M_and_mask(self, reservoir_size, connectivity, spectral_radius):
        M = 2 * (self.rng_M.random((reservoir_size, reservoir_size)) - 0.5)
        sparsification_mask = self.rng_M.binomial(1, connectivity, size=(reservoir_size, reservoir_size))
        original_M = M.copy()
        sparsification_mask_copy = sparsification_mask.copy()
        M *= sparsification_mask
        M *= spectral_radius / max(np.abs(np.linalg.eigvals(M))).real
        return M, original_M, sparsification_mask_copy

    # randomly generate Win so that there is one element in each row
    def generate_Win(self, reservoir_size, input_size):
        Win = np.zeros((reservoir_size, input_size))
        random_cols = self.rng_Win.integers(0, input_size, reservoir_size)
        random_values = self.rng_Win.uniform(-1, 1, reservoir_size)
        Win[np.arange(reservoir_size), random_cols] = random_values
        return Win

    #reservoir flow for open-loop system
    def reservoir_update_equation(self, t, r):
        return self.gamma * (-r + np.tanh(self.M @ r + self.sigma * self.Win @ self.u.state))

    #reservoir flow for closed-loop system
    def reservoir_update_equation_prediction(self, t, r):
        return self.gamma * (-r + np.tanh(self.M @ r + self.sigma * self.Win @ self.phi(self.q(r))))

    # function that performs listening stage by integrating both the drive and response systems in the RC forward for a time t_listen
    def listen(self, t_listen):
        num_timesteps = int(t_listen / self.dt)

        for _ in range(num_timesteps):
            self.u.state = RK4step(self.u, self.u.state, self.t, self.dt)
            self.r = RK4step(self.res_update_func_listen, self.r, self.t, self.dt)
            self.t += self.dt

    # function that performs the training stage by integrating forward the drive response states and collecting response and drive states to fit readout map
    def train(self, t_train):
        num_timesteps = int(t_train / self.dt)

        u_states = []
        symmetry_broken_r_states = []
        for _ in range(num_timesteps):
            self.u.state = RK4step(self.u, self.u.state, self.t, self.dt)
            u_states.append(self.u.state)
            self.r = RK4step(self.res_update_func_listen, self.r, self.t, self.dt)
            symmetry_broken_r_states.append(self.q(self.r))
            self.t += self.dt

        u_states = np.array(u_states)
        symmetry_broken_r_states = np.array(symmetry_broken_r_states)
        self.parameterise_phi(symmetry_broken_r_states, u_states)

    # function that performs prediction stage with the trained RC and returns the predictions/attractor reconstruction for the passed amount of time.
    def predict(self, t_predict):
        num_timesteps = int(t_predict / self.dt)

        predictions = [self.phi(self.q(self.r))]
        for _ in range(num_timesteps):
            self.r = RK4step(self.res_update_func_pred, self.r, self.t, self.dt)
            prediction = self.phi(self.q(self.r))
            predictions.append(prediction)
            self.t += self.dt

        return np.array(predictions)

    # function that takes the synchronised set of response and drive states that are supposed to be fit and finds the parameters of the linear map by ridge regression
    def parameterise_phi(self, X, Y):
        n = X.shape[0]
        dims_x = X.shape[1]
        dims_y = Y.shape[1]
        A = np.linalg.inv(X.T @ X + self.beta * np.identity(dims_x)) @ X.T @ Y
        b = (1 / n) * np.sum(Y - X @ A, axis=0)
        self.A = A
        self.b = b
        self.phi = lambda x: self.A.T @ x + self.b

    # function that changes the spectral radius of M, with a built in consideration for  the setting of rho(M)=0 renders the topology un recoverable so it has a copy system for that occasion
    def set_spectral_radius(self, spectral_radius):
        if spectral_radius == 0:
            self.M = np.zeros_like(self.original_M)
        else:
            self.M = (self.original_M * self.sparsification_mask) * (spectral_radius / max(np.abs(np.linalg.eigvals(self.original_M * self.sparsification_mask))).real)
        self.spectral_radius = spectral_radius

    # function that converts normal time periods to timesteps
    def time_ts(self, natural_time):
        return int(natural_time/self.dt)

    # reinitialise the same RC to be untrained with the same ICs
    def reinitialise_same_RC(self):
        self.reinitialise_state()
        self.u.reinitialise_ic()
        self.A = None
        self.b = None
        self.phi = None

    # reinitialise to a different response state
    def reinitialise_state(self):
        self.r = 2 * (self.rng_state.random(self.reservoir_size) - 0.5)
        self.t = 0.0

    # reinitialise to a different adjacency matrix of the reservoir M
    def reinitialise_M(self):
        self.M, self.original_M, self.sparsification_mask = self.generate_M_and_mask(self.reservoir_size, self.connectivity, self.spectral_radius)

    # reinitialise to a different readin matrix Win
    def reinitialise_Win(self):
        self.Win = self.generate_Win(self.reservoir_size, self.u.state.size)

    # reinitialise the RC random elements to new ones
    def reinitialise_RC(self):
        self.reinitialise_M()
        self.reinitialise_Win()
        self.reinitialise_state()


In [ ]:
# class that takes a reservoir computer object and performs a state space exploration a trained RC by integrating 100 random ICs for a sufficient time
class ReservoirComputer_BulkSimulation(ReservoirComputer):
    def __init__(self, drive_system, reservoir_size, connectivity, spectral_radius, gamma, sigma):
        super().__init__(drive_system, reservoir_size, connectivity, spectral_radius, gamma, sigma)

    def predict_100(self, t_predict):
        predictions_100 = []
        for _ in range(100):
            self.reinitialise_state()
            predictions = self.predict(t_predict)
            predictions_100.append(predictions)

        print("Predictions complete")
        return np.array(predictions_100)

    def simulate_100(self, t_listen, t_train, t_predict):
        self.rng_state = np.random.default_rng(1913)
        self.reinitialise_same_RC()

        self.listen(t_listen)
        self.train(t_train)
        trajectories = self.predict_100(t_predict)
        return np.array(trajectories)